https://github.com/neuml/txtinstruct/blob/master/examples/01_Introducing_txtinstruct.ipynb

In [1]:
!pip install git+https://github.com/neuml/txtai git+https://github.com/neuml/txtinstruct

  Cloning https://github.com/neuml/txtai to /private/var/folders/kt/7gdyyk6d75lbw1xwdqkrbrjw0000gn/T/pip-req-build-hkzry3lc
  Running command git clone --filter=blob:none --quiet https://github.com/neuml/txtai /private/var/folders/kt/7gdyyk6d75lbw1xwdqkrbrjw0000gn/T/pip-req-build-hkzry3lc
  Resolved https://github.com/neuml/txtai to commit 6b4979e670ad90ae3f6aaeda8e6accd993ae96a3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/neuml/txtinstruct to /private/var/folders/kt/7gdyyk6d75lbw1xwdqkrbrjw0000gn/T/pip-req-build-v767uubx
  Running command git clone --filter=blob:none --quiet https://github.com/neuml/txtinstruct /private/var/folders/kt/7gdyyk6d75lbw1xwdqkrbrjw0000gn/T/pip-req-build-v767uubx
  Resolved https://github.com/neuml/txtinstruct to commit eb9fd98e7afe8aa78a10f9f0bc2878f84eba1a30
  Installing build dependencies ... done
  Getting requirements to build wheel ..

In [3]:
pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 3.3 MB/s eta 0:00:004.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:

from datasets import load_dataset

from txtinstruct.models import StatementGenerator

# Load SQuAD dataset
dataset = load_dataset("squad", split="train")

# Train model
generator = StatementGenerator()
model, tokenizer = generator(
    "google/flan-t5-small",
    dataset,
    "sequence-sequence",
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=128 // 16,
    num_train_epochs=0.1,
    logging_steps=100,
)
   

/Users/alprielse/src/sandboxed/2023-05-10_txtinstruct/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 1063.87it/s]


Dataset squad downloaded and prepared to /Users/alprielse/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


Dataset generator downloaded and prepared to /Users/alprielse/.cache/huggingface/datasets/generator/default-4dafeecd279c8f2a/0.0.0. Subsequent calls will reuse this data.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate`: Run `pip install --upgrade accelerate`